In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_DIR = "./drive/MyDrive/NLP/"

In [2]:
# !pip install tqdm boto3 requests regex sentencepiece sacremoses
# !pip install transformers


In [3]:
# load the data
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np

DATA_DIR = ""
fname = "cmv.tar.bz2"
# url = "https://chenhaot.com/data/cmv/" + fname

# download if not exists
# if not os.path.isfile(fname):
#     f = BytesIO()
#     with request.urlopen(url) as resp, open(fname, 'wb') as f_disk:
#         data = resp.read()
#         f_disk.write(data)  # save to disk too
#         f.write(data)
#         f.seek(0)
# else:
f = open(DATA_DIR+fname, 'rb')


tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

all_train_fname = "all/train_period_data.jsonlist.bz2"
all_test_fname = "all/heldout_period_data.jsonlist.bz2"

pair_train_fname = "pair_task/train_pair_data.jsonlist.bz2"
pair_test_fname = "pair_task/train_pair_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)
# all_train_bzlist = tar.extractfile(all_train_fname)
# pair_train_bzlist = tar.extractfile(pair_train_fname)

# Deserialize the JSON list
original_posts_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]
# all_train = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(all_train_bzlist)
# ]
# pair_train = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(pair_train_bzlist)
# ]
test_bzlist = tar.extractfile(test_fname)
# all_test_bzlist = tar.extractfile(all_test_fname)
# pair_test_bzlist = tar.extractfile(pair_test_fname)

original_posts_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(test_bzlist)
]
# all_test = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(all_test_bzlist)
# ]
# pair_test = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(pair_test_bzlist)
# ]
f.close()

In [4]:
malleable_op = [post["name"] for post in original_posts_train if post["delta_label"]]
print(f'{len(original_posts_train)}, {len(malleable_op)}')

10743, 3191


In [5]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score


In [6]:
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
            ]
    return "\n".join(lines)

In [7]:
op_text_train = [
    cleanup(original_post['selftext'])
    # original_post['selftext']  # try this at home
    for original_post
    in original_posts_train
]
op_text_test = [
    cleanup(original_post['selftext'])
    # original_post['selftext']  # try this at home
    for original_post
    in original_posts_test
]

In [8]:
vect = TfidfVectorizer(use_idf=False, norm='l1')
X_train = vect.fit_transform(op_text_train)

In [9]:
lr = LogisticRegressionCV(Cs=10, class_weight='balanced',
                          scoring='roc_auc', solver='sag',
                          tol=0.001, max_iter=500,
                          random_state=0)

In [10]:
X_train.shape

(10743, 48296)

In [11]:
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])

lr.fit(X_train, is_malleable_train)

In [12]:
X_test = vect.transform(op_text_test)
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
test_roc = roc_auc_score(is_malleable_test,lr.predict_proba(X_test)[:,1])
print("Test ROC AUC score: {:.3f}".format(test_roc))

NotFittedError: This LogisticRegressionCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Longformer

In [12]:
import pandas as pd
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [13]:
config = LongformerConfig()

config

LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.17.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [14]:
# load model and tokenizer and define length of the text sequence
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weigh

In [15]:
model.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.17.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [16]:
example_text = op_text_train[0]
long_input = tokenizer(example_text,padding='max_length', max_length = 1024, 
                       truncation=True, return_tensors="pt")

print(long_input['input_ids'])
print(long_input['attention_mask'])

tensor([[  0, 100, 206,  ...,   1,   1,   1]])
tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [17]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [int(label) for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 1024, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [18]:
import pandas as pd
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
df_train = pd.DataFrame(list(zip(op_text_train,is_malleable_train)),columns = ['text','label'])
df_train, df_val = np.split(df_train.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_train))])
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = pd.DataFrame(list(zip(op_text_test,is_malleable_test)),columns = ['text','label'])

In [19]:
print("tokenizing data")
train = Dataset(df_train)
val = Dataset(df_val)

tokenizing data


In [20]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=False,drop_last=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=2,drop_last=True)

In [21]:
class LongClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(LongClassifier, self).__init__()

        self.long = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        out = self.long(input_ids= input_id, attention_mask=mask,return_dict=True)
        logits = out.logits
        out = self.sigmoid(logits)
        out = out[:,1]
        return out

In [22]:
from torch.optim import Adam
from tqdm import tqdm
EPOCHS = 5
model = LongClassifier()
LR = 1e-6 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= LR)

if use_cuda:

        model = model.cuda()
        criterion = criterion.cuda()
print("start training")

best_val_loss = 100
for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output.squeeze(), train_label.float())
            total_loss_train += batch_loss.item()
            
            acc = (torch.round(output) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output.squeeze(), val_label.float())
                total_loss_val += batch_loss.item()
                
                acc = (torch.round(output) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
            | Train Accuracy: {total_acc_train / len(train): .3f} \
            | Val Loss: {total_loss_val / len(val): .3f} \
            | Val Accuracy: {total_acc_val / len(val): .3f}')
        if (total_loss_val/len(val))<best_val_loss:
            best_val_loss = total_loss_val/len(val)
            print("New best val loss, model saved!")
            torch.save(model,'best_model')
                  
  

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weigh

start training


100%|██████████| 4297/4297 [32:56<00:00,  2.17it/s]


Epochs: 1 | Train Loss:  0.306             | Train Accuracy:  0.700             | Val Loss:  0.297             | Val Accuracy:  0.714
New best val loss, model saved!


100%|██████████| 4297/4297 [32:06<00:00,  2.23it/s]


...

Epochs: 3 | Train Loss:  0.278             | Train Accuracy:  0.710             | Val Loss:  0.305             | Val Accuracy:  0.713


100%|██████████| 4297/4297 [32:05<00:00,  2.23it/s]


Epochs: 4 | Train Loss:  0.236             | Train Accuracy:  0.765             | Val Loss:  0.333             | Val Accuracy:  0.671


100%|██████████| 4297/4297 [32:00<00:00,  2.24it/s]


Epochs: 5 | Train Loss:  0.164             | Train Accuracy:  0.862             | Val Loss:  0.408             | Val Accuracy:  0.662


In [23]:
test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

In [24]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    model = model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in test_dataloader:

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)

          acc = (torch.round(output) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

Test Accuracy:  0.565


In [25]:
best_model = torch.load('best_model')


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    best_model = best_model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in test_dataloader:

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = best_model(input_id, mask)

          acc = (torch.round(output) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

Test Accuracy:  0.560


## Previous try with BERT base

In [ ]:
import torch
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', './test/bert_saved_model/')  # E.g. tokenizer was saved using `save_pretrained('./test/saved_model/')`

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = op_text_train[0]
bert_input = tokenizer(example_text,padding='max_length', max_length = 5000, 
                       truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101,  146, 1341,  ...,    0,    0,    0]])
tensor([[0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [ ]:
lengths = np.array([len(text.split()) for text in op_text_train])
print(lengths.max())
print(lengths.argmax())
print(op_text_train[224]) # Longest OP text

5511
224
## Section I: Definitions of "Social Democracy" and "Socialism"

Social democracy "is defined as a policy regime involving a universal welfare state and collective bargaining schemes within the framework of a capitalist economy. It is often used in this manner to refer to the social models and economic policies prominent in Western and Northern Europe during the later half of the 20th century."

I really like the political economic structures of Scandinavian nations. The basic idea of a "social democracy" is that "the means of production are mainly under private ownership; that markets remain the dominant form of economic coordination; and that profit-seeking self-interested enterprises and the accumulation of capital remains the fundamental driving force behind economic activity. However, unlike a laissez-faire economy, the government weilds considerable indirect influence over the economy through fiscal and monetary policies designed to counteract economic downturns and capi

In [ ]:
print(np.sort(lengths)[-5000:])

[ 201  201  201 ... 2429 3595 5511]


In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] I think the world is automating fast enough that a utopia will arise where no one will have to work anymore. Within the next 2 decades or so, having a job won't mean much, and most people will be artists and scientists. My parents let me live with them, so I can just wait until the utopia happens. CMV. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [int(label) for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
import pandas as pd
df_train = pd.DataFrame(list(zip(op_text_train,is_malleable_train)),columns = ['text','label'])
df_train, df_val = np.split(df_train.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_train))])
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = pd.DataFrame(list(zip(op_text_test,is_malleable_test)),columns = ['text','label'])

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

In [ ]:
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])

In [ ]:
print("tokenizing data")
train = Dataset(df_train)
val = Dataset(df_val)

tokenizing data


In [ ]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=False)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

In [ ]:
from torch.optim import Adam
from tqdm import tqdm
EPOCHS = 5
model = BertClassifier()
LR = 1e-6 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= LR)

if use_cuda:

        model = model.cuda()
        criterion = criterion.cuda()
print("start training")
for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output.squeeze(), train_label.float())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output.squeeze(), val_label.float())
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
            | Train Accuracy: {total_acc_train / len(train): .3f} \
            | Val Loss: {total_loss_val / len(val): .3f} \
            | Val Accuracy: {total_acc_val / len(val): .3f}')
                  
  

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.4.attention.out_lin.weight', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'distilbert.transformer.layer.5.attention.k_lin.weight', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.3.attention.q_lin.weight', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.5.output_layer_norm.bias', 'distilbert.transformer.layer.1.output_layer_norm.bias', 'distilbert.transformer.layer.2.sa_layer_norm.bias', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.tran

start training


  0%|          | 13/4297 [02:55<16:03:29, 13.49s/it]


KeyboardInterrupt: ignored

In [ ]:
test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

In [ ]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    model = model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in test_dataloader:

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)

          acc = (output.argmax(dim=1) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

In [ ]:
def top_features(coef, words, n=15):
    order = np.argsort(-coef)
    for i_hi, i_lo in zip(order[:n], order[:-n - 1:-1]):
        row_vals = coef[i_hi], words[i_hi], coef[i_lo], words[i_lo]
        print("[{:>4.1f}] {:<20}[{:>4.1f}] {:<20}".format(*row_vals))

In [ ]:
top_features(lr.coef_.flatten(), vect.get_feature_names_out())